In [553]:
import pandas as pd
import numpy as np
import requests as rq
import cv2 as cv
from PIL import Image

In [554]:
# car_haar_cascade = "C:\\Coding\\GitHub\\Billboard-Car-Count\\car_detection_haar.xml"

traffic_video_one = "C:\\Coding\\GitHub\\Billboard-Car-Count\\media\\traffic_one.mp4"
traffic_video_two = "C:\\Coding\\GitHub\\Billboard-Car-Count\\media\\traffic_two.mp4"
traffic_video_three = "C:\\Coding\\GitHub\\Billboard-Car-Count\\media\\traffic_three.mp4"
traffic_video_four = "C:\\Coding\\GitHub\\Billboard-Car-Count\\media\\traffic_four.mp4"

video_height = 360
video_width = 640

In [555]:
carscrossedup = 0  # keeps track of cars that crossed up
carscrosseddown = 0  # keeps track of cars that crossed down

frame_number = 0
cars_crossed_line = 0
total_cars_seen = 0

car_ids = []
car_ids_crossed = []

In [556]:
video_capture = cv.VideoCapture(traffic_video_four)

# The numbers in set are property identifiers 3 = width, 4 = height
# This sets video height and width to predetermined height and width
video_capture.set(3, video_width)
video_capture.set(4, video_height)

# car_cascade_classifier = cv.CascadeClassifier(car_haar_cascade)

# 0x7634706d is used to force videowriter to output mp4 if other conflicts exist
video_result = cv.VideoWriter('video_result.mp4', 0x7634706d, 15.0, (video_width, video_height))

background_subtractor = cv.createBackgroundSubtractorKNN()


In [557]:
# video_result = cv.VideoWriter('video_result.mp4', 0x7634706d, 15.0, (video_width, video_height))

# while True:
#     return_val, captured_image = video_capture.read()

#     if (type(captured_image) == type(None)):
#         break

#     to_grayscale = cv.cvtColor(captured_image, cv.COLOR_BGR2GRAY)

#     car_detected = car_cascade_classifier.detectMultiScale(to_grayscale, 1.1, 2)

#     for (x_cord, y_cord, width, height) in car_detected:
#         cv.rectangle(captured_image, (x_cord, y_cord), (x_cord+width, y_cord+height), (0, 255, 255), 2)

#     video_result.write(captured_image)
# video_result.release()
# cv.destroyAllWindows()

In [558]:

video_result = cv.VideoWriter('video_result.mp4', 0x7634706d, 15.0, (video_width, video_height), isColor=False)

while True:
    return_val, captured_image = video_capture.read()

    if (type(captured_image) == type(None)):
        break

    # Changes image to grayscale
    gray_image = cv.cvtColor(captured_image, cv.COLOR_BGR2GRAY)

    foreground_mask = background_subtractor.apply(gray_image, None, .01)

    # Slight blur to eliminate some nosie
    grayed_blured_image = cv.blur(foreground_mask, (3, 3))

    # Removes grays and returns black and white (binary) image
    return_val, binary_image = cv.threshold(grayed_blured_image, 80, 255, cv.THRESH_BINARY)

##############



    

    video_result.write(binary_image)
video_result.release()
cv.destroyAllWindows()